# shp

> Spatially Homogenious Pixels Identification

In [ ]:
#| default_exp shp

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from scipy import stats
import numpy as np
from moraine.utils_ import is_cuda_available
if is_cuda_available():
    import cupy as cp
import itertools

In [ ]:
#| export
import numpy as np
from moraine.utils_ import is_cuda_available, get_array_module
if is_cuda_available():
    import cupy as cp
import math
import numba
from numba import prange
from moraine.utils_ import ngpjit, ngjit

## Kolmogorov-Smirnov (KS) two-sample test

In [ ]:
#| export
@ngjit
def _ks_p_numba(x):
    x2 = -2*x*x
    p = 0.0
    p2 = 0.0
    sign = 1
    for i in range(1,101):
        p += sign*2*math.exp(x2*i*i)
        if (p == p2):
            return p
        else:
            sign = -sign
            p2 = p
    return p

In [ ]:
#| export
@ngjit
def _ks_d_numba(ref, sec, n_imag):
    j1 = 0; j2 = 0; d = 0.0; dmax = 0.0
    while (j1 < n_imag) and (j2 < n_imag):
        f1 = ref[j1]; f2 = sec[j2]
        if f1 <= f2: j1 += 1;
        if f1 >= f2: j2 += 1;
        d = abs((j2-j1)/n_imag)
        if (d > dmax): dmax = d
    return dmax

In [ ]:
#| export
@ngpjit
def _ks_test_no_dist_numba(
    rmli, # sorted rmli stack
    az_half_win,
    r_half_win,
):
    n_az, n_r,n_imag = rmli.shape
    p = np.empty((n_az,n_r,2*az_half_win+1,2*r_half_win+1),dtype=rmli.dtype)
    for i in prange(n_az):
        for j in prange(n_r):
            ref_rmli = rmli[i,j]
            if math.isnan(ref_rmli[-1]):
                for l in range(-az_half_win, az_half_win+1):
                    for m in range(-r_half_win,r_half_win+1):
                        p[i,j,l+az_half_win,m+r_half_win] = np.nan
            else:
                for l in range(-az_half_win, az_half_win+1):
                    for m in range(-r_half_win,r_half_win+1):
                        sec_i = i+l
                        sec_j = j+m
                        if (sec_i<0) or (sec_i>=n_az) or (sec_j<0) or (sec_j>=n_r):
                            p[i,j,l+az_half_win,m+r_half_win] = np.nan
                        else:
                            sec_rmli = rmli[sec_i, sec_j]
                            if math.isnan(sec_rmli[-1]):
                                p[i,j,l+az_half_win,m+r_half_win] = np.nan
                            else:
                                dmax = _ks_d_numba(ref_rmli, sec_rmli, n_imag)
                                en = math.sqrt(n_imag/2)
                                p[i,j,l+az_half_win,m+r_half_win] = _ks_p_numba((en+0.12+0.11/en)*dmax)
    return p

@ngpjit
def _ks_test_numba(
    rmli, # sorted rmli stack
    az_half_win,
    r_half_win,
):
    n_az, n_r,n_imag = rmli.shape
    dist = np.empty((n_az,n_r,2*az_half_win+1,2*r_half_win+1),dtype=rmli.dtype)
    p = np.empty((n_az,n_r,2*az_half_win+1,2*r_half_win+1),dtype=rmli.dtype)
    for i in prange(n_az):
        for j in prange(n_r):
            ref_rmli = rmli[i,j]
            if math.isnan(ref_rmli[-1]):
                for l in range(-az_half_win, az_half_win+1):
                    for m in range(-r_half_win,r_half_win+1):
                        dist[i,j,l+az_half_win,m+r_half_win] = np.nan
                        p[i,j,l+az_half_win,m+r_half_win] = np.nan
            else:
                for l in range(-az_half_win, az_half_win+1):
                    for m in range(-r_half_win,r_half_win+1):
                        sec_i = i+l
                        sec_j = j+m
                        if (sec_i<0) or (sec_i>=n_az) or (sec_j<0) or (sec_j>=n_r):
                            dist[i,j,l+az_half_win,m+r_half_win] = np.nan
                            p[i,j,l+az_half_win,m+r_half_win] = np.nan
                        else:
                            sec_rmli = rmli[sec_i, sec_j]
                            if math.isnan(sec_rmli[-1]):
                                p[i,j,l+az_half_win,m+r_half_win] = np.nan
                            else:
                                dmax = _ks_d_numba(ref_rmli, sec_rmli, n_imag)
                                en = math.sqrt(n_imag/2)
                                dist[i,j,l+az_half_win,m+r_half_win] = dmax
                                p[i,j,l+az_half_win,m+r_half_win] = _ks_p_numba((en+0.12+0.11/en)*dmax)
    return dist, p

In [ ]:
#| export
@ngpjit
def _sort_numba(
    rmli, # rmli stack
):
    n_az, n_r,n_imag = rmli.shape
    sorted_rmli = np.empty_like(rmli)
    for i in prange(n_az):
        for j in prange(n_r):
            sorted_rmli[i,j] = np.sort(rmli[i,j])
    return sorted_rmli

In [ ]:
#| export
# It looks cupy do not support pointer to pointer: float** rmli_stack
if is_cuda_available():
    _ks_test_kernel = cp.ElementwiseKernel(
        'raw T rmli_stack, int32 nlines, int32 width, int32 nimages, int32 az_half_win, int32 r_half_win',
        'raw T dist, raw T p',
        '''
        int az_win = 2*az_half_win+1;
        int r_win = 2*r_half_win+1;
        int win = az_win*r_win;
        
        int ref_idx = i/win;
        int ref_az = ref_idx/width;
        int ref_r = ref_idx -ref_az*width;
        
        int win_idx = i - ref_idx*win;
        int win_az = win_idx/r_win;
        int win_r = win_idx - win_az*r_win;
        int sec_az = ref_az + win_az - az_half_win;
        int sec_r = ref_r + win_r - r_half_win;
        int sec_idx = sec_az*width + sec_r;
        
        if (ref_r >= width && ref_az >= nlines) {
            return;
        }
        if (sec_az < 0 || sec_az >= nlines || sec_r < 0 || sec_r >= width) {
            dist[ref_idx*win+win_az*r_win+win_r] = CUDART_NAN;
            p[ref_idx*win+win_az*r_win+win_r] = CUDART_NAN;
            return;
        }

        // if data contain nan value, nan are sorted to the end
        if (isnan(rmli_stack[ref_idx*nimages + nimages-1]) || isnan(rmli_stack[sec_idx*nimages + nimages-1])) {
            dist[ref_idx*win+win_az*r_win+win_r] = CUDART_NAN;
            p[ref_idx*win+win_az*r_win+win_r] = CUDART_NAN;
            return;
        }

        // Compute the maximum difference between the cumulative distributions
        int j1 = 0, j2 = 0;
        T f1, f2, d, dmax = 0.0, en = nimages;
    
        while (j1 < nimages && j2 < nimages) {
            f1 = rmli_stack[ref_idx*nimages + j1];
            f2 = rmli_stack[sec_idx*nimages + j2];
            if (f1 <= f2) j1++;
            if (f1 >= f2) j2++;
            d = fabs((j2-j1)/en);
            if (d > dmax) dmax = d;
        }
        en=sqrt(en/2);
        p[ref_idx*win+win_az*r_win+win_r] = ks_p((en+0.12+0.11/en)*dmax);
        dist[ref_idx*win+win_az*r_win+win_r] = dmax;
        ''',
        name = 'ks_test_kernel',no_return=True,
        preamble = '''
        #include <cupy/math_constants.h>
        __device__ T ks_p(T x)
        {
            T x2 = -2.0*x*x;
            int sign = 1;
            T p = 0.0,p2 = 0.0;
        
            for (int i = 1; i <= 100; i++) {
                p += sign*2*exp(x2*i*i);
                if (p==p2) return p;
                sign = -sign;
                p2 = p;
            }
            return p;
        }
        ''',)
    _ks_test_no_dist_kernel = cp.ElementwiseKernel(
        'raw T rmli_stack, int32 nlines, int32 width, int32 nimages, int32 az_half_win, int32 r_half_win',
        'raw T p',
        '''
        int az_win = 2*az_half_win+1;
        int r_win = 2*r_half_win+1;
        int win = az_win*r_win;
        
        int ref_idx = i/win;
        int ref_az = ref_idx/width;
        int ref_r = ref_idx -ref_az*width;
        
        int win_idx = i - ref_idx*win;
        int win_az = win_idx/r_win;
        int win_r = win_idx - win_az*r_win;
        int sec_az = ref_az + win_az - az_half_win;
        int sec_r = ref_r + win_r - r_half_win;
        int sec_idx = sec_az*width + sec_r;
        
        if (ref_r >= width && ref_az >= nlines) {
            return;
        }
        if (sec_az < 0 || sec_az >= nlines || sec_r < 0 || sec_r >= width) {
            p[ref_idx*win+win_az*r_win+win_r] = CUDART_NAN;
            return;
        }

        // if data contain nan value, nan are sorted to the end
        if (isnan(rmli_stack[ref_idx*nimages + nimages-1]) || isnan(rmli_stack[sec_idx*nimages + nimages-1])) {
            p[ref_idx*win+win_az*r_win+win_r] = CUDART_NAN;
            return;
        }
        
        // Compute the maximum difference between the cumulative distributions
        int j1 = 0, j2 = 0;
        T f1, f2, d, dmax = 0.0, en = nimages;
        
        while (j1 < nimages && j2 < nimages) {
            f1 = rmli_stack[ref_idx*nimages + j1];
            f2 = rmli_stack[sec_idx*nimages + j2];
            // check if there nan value in the data

            if (f1 <= f2) j1++;
            if (f1 >= f2) j2++;
            d = fabs((j2-j1)/en);
            if (d > dmax) dmax = d;
        }
        en=sqrt(en/2);
        p[ref_idx*win+win_az*r_win+win_r] = ks_p((en+0.12+0.11/en)*dmax);
        ''',
        name = 'ks_test_no_dist_kernel',no_return=True,
        preamble = '''
        #include <cupy/math_constants.h>
        __device__ T ks_p(T x)
        {
            T x2 = -2.0*x*x;
            int sign = 1;
            T p = 0.0,p2 = 0.0;
        
            for (int i = 1; i <= 100; i++) {
                p += sign*2*exp(x2*i*i);
                if (p==p2) return p;
                sign = -sign;
                p2 = p;
            }
            return p;
        }
        ''',)

In [ ]:
#| export
def ks_test(rmli:np.ndarray, # the rmli stack, dtype: cupy.floating
            az_half_win:int, # SHP identification half search window size in azimuth direction
            r_half_win:int, # SHP identification half search window size in range direction
            block_size:int=128, # the CUDA block size, it only affects the calculation speed, only applied if input is cupy.ndarray
            return_dist:bool=False, # if return the KS test statistics `dist`
           ) -> tuple[np.ndarray,np.ndarray] : # if return_dist == True, return `dist` and p value `p`. Otherwise, only `p` is returned.
    '''
    SHP identification based on Two-Sample Kolmogorov-Smirnov Test.
    '''
    xp = get_array_module(rmli)
    az_win = 2*az_half_win+1
    r_win = 2*r_half_win+1
    nlines, width, nimages = rmli.shape
    if xp is np:
        sorted_rmli = _sort_numba(rmli)
        if return_dist:
            return _ks_test_numba(sorted_rmli, az_half_win, r_half_win)
        else:
            return _ks_test_no_dist_numba(sorted_rmli, az_half_win, r_half_win)

    else:
        sorted_rmli = cp.sort(rmli,axis=-1)
        if return_dist:
            dist = cp.empty((nlines,width,az_win,r_win),dtype=rmli.dtype)
            p = cp.empty((nlines,width,az_win,r_win),dtype=rmli.dtype)
        
            _ks_test_kernel(sorted_rmli,cp.int32(nlines),cp.int32(width),cp.int32(nimages),
                            cp.int32(az_half_win),cp.int32(r_half_win),dist,p,
                            size=width*nlines*r_win*az_win,block_size=block_size)
            return dist,p
        else:
            p = cp.empty((nlines,width,az_win,r_win),dtype=rmli.dtype)
        
            _ks_test_no_dist_kernel(sorted_rmli,cp.int32(nlines),cp.int32(width),cp.int32(nimages),
                            cp.int32(az_half_win),cp.int32(r_half_win),p,
                            size=width*nlines*r_win*az_win,block_size=block_size)
            return p

The `ks_test` function apply the Two-Sample Kolmogorov-Smirnov Test on a stack of rmli images to identify SHPs candidate for further processing. This method is originally published in [@ferrettiNewAlgorithmProcessing2011]. This function is designed to run on GPU for high speed.

The `rmli` is a three dimentional cupy/numpy `ndarray`. The `dtype` should be `float`. From outerest to innerest, the three dimentions are azimuth, range and image. For each pixel P, a search window centered at P is defined by `az_half_win` and `r_half_win`. All pixels in this search window is compared with P by KS test. They are refered here as secondary pixels. The total number of secondary pixels (including P) is (2\*`az_half_win`+1)\*(2\*`r_half_win`+1).

The returns are the ks test statistic which is the maximum value of the absolute difference between the emperical cumulative distribution functions of the two samples, and p value. Both of them are 4 dimentional cupy/numpy ndarrays. From outerest ot innerest, they are azimuth, range, secondary pixel relative azimuth, secondary pixel relative range. For P at the corner of the image where part of the search window is out of the image, the result is `nan`.

Here is a simplest example. First simulate rmli time series of two pixels from two correlated normal distributions:

In [ ]:
sample_size = 20
rng = np.random.default_rng()
sample1 = stats.uniform.rvs(size=sample_size, random_state=rng).astype(np.float32)
sample2 = stats.norm.rvs(size=sample_size, random_state=rng).astype(np.float32)

rmli_stack = np.stack((np.asarray(sample1), np.asarray(sample2))).reshape(1,2,sample_size)
rmli_stack.shape

(1, 2, 20)

The shape of `rmli_stack` shows it contains 20 images. Each of the image has 1 pixel in azimuth dimention and 2 pixels in range dimention. Set the `az_half_win` and `r_half_win` to 1 and apply the `ks_test` function:

In [ ]:
dist,p = ks_test(rmli_stack,1,1,return_dist=True)
print(dist.shape)
print(dist)

(1, 2, 3, 3)
[[[[nan nan nan]
   [nan 0.  0.5]
   [nan nan nan]]

  [[nan nan nan]
   [0.5 0.  nan]
   [nan nan nan]]]]


`dist` is the ks test statistic. The shape of it shows for each pixel P in this `1*2` image, a `3*3` search window is defined and all pixels in this search window is test with P. The value `0` in `dist` is the ks test result of pixel P and pixel P itself. The value `nan` means the secondary pixel is out of the image and no ks test is applied.

In [ ]:
print(p.shape)
print(p)

(1, 2, 3, 3)
[[[[       nan        nan        nan]
   [       nan 0.         0.00816168]
   [       nan        nan        nan]]

  [[       nan        nan        nan]
   [0.00816168 0.                nan]
   [       nan        nan        nan]]]]


`p` is the ks test p value with same shape of `dist`.

In [ ]:
print(stats.ks_2samp(sample1, sample2,method='asymp'))

KstestResult(statistic=0.5, pvalue=0.00777741, statistic_location=-0.015991585, statistic_sign=-1)


By comparing the result of `ks_test` and `ks_2samp` from `scipy`, the statistics are same which prove the correctness of `ks_test`. The difference in p value is because the approcimation method used are different but the orders of magnitudes are consistent.

`ks_test` also accept cupy array:

In [ ]:
if is_cuda_available():
    rmli_stack_cp = cp.asarray(rmli_stack)
    dist_cp,p_cp = ks_test(rmli_stack_cp,1,1,return_dist=True)
    np.testing.assert_array_equal(dist,cp.asnumpy(dist_cp))
    np.testing.assert_array_almost_equal(p,cp.asnumpy(p_cp))

In [ ]:
#| hide
# test
nimages = 5
nlines = 5
width = 5
az_half_win = 5
r_half_win = 5

rng = np.random.default_rng()
sample_list = []
for i in range(nlines*width):
    if i==0:
        sample_list.append(np.sort(stats.uniform.rvs(size=nimages, random_state=rng)).astype(np.float32))
    else:
        sample_list.append(np.sort(stats.norm.rvs(size=nimages, random_state=rng)).astype(np.float32))
    
sample_stack = np.stack(sample_list).reshape(nlines,width,nimages)
dist, p = ks_test(sample_stack,az_half_win,r_half_win,return_dist=True)
p_ = ks_test(sample_stack,az_half_win,r_half_win,return_dist=False)
np.testing.assert_array_equal(p, p_)
assert dist.shape == (nlines,width,az_half_win*2+1,r_half_win*2+1)
for az,r,az_win,r_win in itertools.product(range(nlines),range(width),range(az_half_win),range(r_half_win)):
    sec_az = az + az_win - az_half_win
    sec_r = r + r_win - r_half_win
    if (sec_az<0 or sec_az>nlines or sec_r<0 or sec_r>width):
        assert np.isnan(dist[az,r,az_win,r_win])
    else:
        scipy_dist,scipy_p = stats.ks_2samp(sample_stack[az,r,:],sample_stack[sec_az,sec_r,:],method='asymp')
        assert abs(dist[az,r,az_win,r_win]-scipy_dist) < 1.0e-7

if is_cuda_available():
    cp_sample_stack = cp.asarray(sample_stack)
    dist_cp,p_cp = ks_test(cp_sample_stack,az_half_win,r_half_win,return_dist=True)
    np.testing.assert_array_equal(dist,cp.asnumpy(dist_cp))
    np.testing.assert_array_almost_equal(p, cp.asnumpy(p_cp))

    # dist, p = cp.asnumpy(dist), cp.asnumpy(p)
    # # we do not test the calculated p value just because 
    # # the p value calculation methods in scipy and numerical recipe
    # # are different and their difference can reach to 10 times!
    # assert dist.shape == (nlines,width,az_half_win*2+1,r_half_win*2+1)
    # for az,r,az_win,r_win in itertools.product(range(nlines),range(width),range(az_half_win),range(r_half_win)):
    #     sec_az = az + az_win - az_half_win
    #     sec_r = r + r_win - r_half_win
    #     if (sec_az<0 or sec_az>nlines or sec_r<0 or sec_r>width):
    #         assert np.isnan(dist[az,r,az_win,r_win])
    #     else:
    #         scipy_dist,p = stats.ks_2samp(sample_stack[az,r,:],sample_stack[sec_az,sec_r,:],method='asymp')
    #         assert abs(dist[az,r,az_win,r_win]-scipy_dist) < 1.0e-7

In [ ]:
#| export
@ngpjit
def select_shp(
    p, # 4D (n_az,n_r,az_win,r_win)
    p_max,
):
    p_shape = p.shape
    is_shp = np.empty(p_shape, dtype=np.bool_)
    shp_num = np.zeros(p_shape[:2], dtype=np.int32)
    for i in prange(p_shape[0]):
        for j in range(p_shape[1]):
            for k in range(p_shape[2]):
                for l in range(p_shape[3]):
                    is_shp[i,j,k,l] = p[i,j,k,l] < p_max
                    if is_shp[i,j,k,l]:
                        shp_num[i,j] += 1
    return is_shp, shp_num

In [ ]:
p = np.random.rand(100*100*5*5).astype(np.float32)
gix = np.random.choice(100*100*5*5,size=1000,replace=False)
p[gix] = np.nan
p = p.reshape(100,100,5,5)
p = (p+np.transpose(p,axes=(0,1,3,2)))/2
for i in range(5):
    p[:,:,i,i] = 1

In [ ]:
is_shp, shp_num = select_shp(p, 0.05)
np.testing.assert_array_equal(is_shp,p< 0.05)
np.testing.assert_array_equal(shp_num, np.count_nonzero(p < 0.05, axis=(-2,-1)).astype(np.int32))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()